In [ ]:
from mesmerize_core import *
from mesmerize_viz import *
import numpy as np
from copy import deepcopy
import pandas as pd

In [ ]:
pd.options.display.max_colwidth = 120

# Paths

`set_parent_raw_data_path()` - sets the **top level raw data directory**

For example, if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

Set `/data/my_name` as the "parent raw data path". This allows you to move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

Set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

**Note: Even on windows just use `/`, you do not have to worry about the issue of `\\` and `\` on windows if you use `pathlib` :D**

In [ ]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

In [ ]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch5/batch.pickle")

**Note: We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

### Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [ ]:
# create a new batch
# df = create_batch(batch_path)
# to load existing batches use `load_batch()`
df = load_batch(batch_path)

### View the dataframe

It is empty with the appropriate columns for mesmerize

In [ ]:
df

# Motion Correction

### Get raw movie

An input movie must be anywhere within `raw data path` or `batch path`. We will use the Sue 2p example.

In [ ]:
# We'll use the Sue movie from caiman
# download it if you don't have it
from caiman.utils.utils import download_demo
download_demo()

In [ ]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/Sue_2x_3000_40_-46.tif")

### Motion correction parameters

Parameters for all algos have the following structure:

```python
{"main": {... params directly passed to caiman}}
```

In [ ]:
# We will start with one version of parameters
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [24, 24],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

### Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [ ]:
# add an item to the batch
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=movie_path,
    params=mcorr_params1,
    item_name=movie_path.stem,  # filename of the movie, but can be anything
)

df

### Run item :D 

In [ ]:
# iloc is used to index rows in the dataframe
df.iloc[0].caiman.run()

In [ ]:
df = df.caiman.reload_from_disk()
df

# Visualize :D 

In [ ]:
mcorr_widget = df.mcorr.viz(data=["input", "mcorr"])
mcorr_widget.show()